### Reading output data from experiment and processing it to analysis and sampling it (to balance data according to parameters) 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy as sp
import os
import json
from datetime import date
import numpy as np
import math
from scipy import stats
from datetime import datetime
from dateutil.parser import parse
from datetime import date

In [2]:
# Sequence bin size
BINSIZE = 2

In [3]:
# Loading the file with all data output (including short video)
directory='data'  
filename = 'allDataMay.csv'
file = os.path.join(directory, filename)
dataRaw = pd.read_csv(file)

# Loading the file with all classified channels
filename = 'classificationDataMay.csv'
file = os.path.join(directory, filename)
dataClas = pd.read_csv(file)

# Merging data with classification 
dataYT = pd.merge(dataRaw, dataClas[['channel_id', 'classification']], on=['channel_id'], how='inner')
dataYT.head()

/Users/larissapspinelli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Unnamed: 0 category   channel                channel_id clar_link  clarify  \
0         2.0     none  NBC News  UCeY0bbntWzzVIaj2z3QigXg       NaN        1   
1         2.0     none  NBC News  UCeY0bbntWzzVIaj2z3QigXg       NaN       -1   
2        14.0     none  NBC News  UCeY0bbntWzzVIaj2z3QigXg       NaN       -1   
3         3.0     none  NBC News  UCeY0bbntWzzVIaj2z3QigXg       NaN       -1   
4         2.0     none  NBC News  UCeY0bbntWzzVIaj2z3QigXg       NaN       -1   

         date                    datePublish  depth  dislikes       ...        \
0  2019-04-14      Published on Mar 29, 2018   21.0      77.0       ...         
1  2019-02-06       Published on Oct 6, 2017   21.0     121.0       ...         
2  2018-12-07      Published on Nov 14, 2018    9.0      94.0       ...         
3  2019-01-29  Started streaming 2 hours ago   20.0      -1.0       ...         
4  2019-02-25      Published on Nov 15, 2016   21.0     176.0       ...         

  rec_qt  recommendations response                  search  setting  \
0    1.0  ['IAvgXmM7Elg']        L           North%20Korea   logged   
1    1.0  ['wqWMrUH5TuU']        L  Las%20Vegas%20shooting   normal   
2    1.0  ['eZDfztfau9A']        L      Hurricane%20Harvey  private   
3    1.0  ['iGn7LTXgcyY']        L  Las%20Vegas%20shooting  private   
4    1.0  ['5QY5pLQqIYM']        0                    DACA      tor   

  subscribers                                              title  verified  \
0         1.0  TurboTax Free 2019 Commercial "Spelling Bee" (...  Verified   
1         1.0  Remembering The Las Vegas Shooting Victims | N...  Verified   
2       978.0  Former First Lady Michelle Obama Describes Lif...       NaN   
3         1.0  Watch Live: FBI, CIA Directors Testify On 'Wor...  Verified   
4         1.0  President Obama To Donald Trump On DACA: Think...       NaN   

      views  classification  
0  193871.0             1.0  
1  109732.0             1.0  
2  101891.0             1.0  
3    2154.0             1.0  
4   51976.0             1.0  

[5 rows x 29 columns]

In [4]:
### 
# Auxiliary fuction to data cleaning/pre-processing
###

# converting sequence extracting []
def auxNum(strNum):
    try:
        return int(strNum[1:-1])
    except:
        return int(0)

# converting number with scale symbols    
def numConv(number):
    try:
        if "K" in number:
            tmp = number.split('K')
            return int(float(tmp[0])*1000)
        elif "M" in number:
            tmp = number.split('M')
            return int(float(tmp[0])*1000000)
        else :
            try:
                tmp = int(number)
                return tmp
            except:
                return 0
    except:
        try:
            return int(number)
        except:
                return 0 
            
# converting time to seconds            
def timeConv(number):
    try:
        if ":" in number:
            #print(number)
            tmp = number.split(':')
            num = 0
            bas = 1
            for i in range(len(tmp)):
                #print(i)
                num = num + int(tmp[len(tmp)-i-1])*bas
                bas = bas*60

            return int(num)

        else :
            try:
                tmp = int(number)
                return tmp
            except:
                return 0
    except:
        try:
            return int(number)
        except:
                return 0                

# getting the day difference of two dates
def daysDiff(date1, date0):
    try:
        if "minutes ago" in date1:
            return 1
        else :
            date0 = parse(date0)
            date1 = parse(date1)
    
        date2 = date1.date()
        date00 = date0.date()
        return (date00-date2).days+1
    except:
        return 1
    
# division of two values     
def divNum(val1, val2):
    try:
        val1 = int(val1)
        val2 = int(val2)
        if val1 < 1 or val2 < 1:
            return 0
        else: 
            return float(val1/val2)
    except:
        return 0
    
# adjusting classification values to [-1, 0, 1] scale  
def auxClas(num):
    if num <= 2:
        return -1
    if num > 3:
        return 1
    else:
        return 0    
    

In [5]:
### 
# Data cleaning
###
    
dataYT['response'] = np.where((dataYT['response']== '0'), 'top movie', dataYT['response'])
dataYT['response'] = np.where((dataYT['response']== 'L'), 'bottom movie', dataYT['response'])
dataYT['response'] = np.where((dataYT['response']== 'R'), 'random movie', dataYT['response'])
dataYT['ord'] = dataYT.apply(lambda x: auxNum(x['order']), axis=1)

dataYT['verify'] = np.where(dataYT['verified']=="Verified", 1, dataYT['verified'])
dataYT['verify'] = np.where(dataYT['verify']=="Official Artist Channel", 1, dataYT['verify'])
dataYT['verify'] = np.where(dataYT['verify']==1, 1, 0)

dataYT['datePublish']  = dataYT.datePublish.astype(str)
dataYT['datePublish'] = dataYT.datePublish.replace({"Published on ": ""}, regex=True)
dataYT['datePublish'] = dataYT.datePublish.replace({"Streamed live on ": ""}, regex=True)
dataYT['datePublish'] = dataYT.datePublish.replace({"Started streaming ": ""}, regex=True)
dataYT['datePublish'] = np.where(dataYT['datePublish'].str.contains("minutes ago", case=False, na=False), dataYT['date'], dataYT['datePublish'])
dataYT['dateDays'] = dataYT.apply(lambda x: daysDiff(x['datePublish'], x['date']), axis=1)

# drop channels not possible to classify and changing classification range
dataYT = dataYT[dataYT['classification']!= 0]
dataYT['clas'] = dataYT.apply(lambda x: auxClas(x['classification']), axis=1)

dataYT = dataYT.replace(to_replace='\%20', value=' ', regex=True)

dataYT.fillna(1)
dataYT['likes'] = dataYT.apply(lambda x: numConv(x['likes']), axis=1)
dataYT['likes'] = dataYT.likes.astype(int)
dataYT['dislikes'] = dataYT.apply(lambda x: numConv(x['dislikes']), axis=1)
dataYT['dislikes'] = dataYT.dislikes.astype(int)
dataYT['views'] = dataYT.apply(lambda x: numConv(x['views']), axis=1)
dataYT['views'] = dataYT.views.astype(int)
dataYT['subscribers'] = dataYT.apply(lambda x: numConv(x['subscribers']), axis=1)
dataYT['subscribers'] = dataYT.subscribers.astype(int)
dataYT['lenght'] = dataYT.apply(lambda x: timeConv(x['lenght']), axis=1)
dataYT['lenght'] = dataYT.lenght.astype(int)

# Relative metrics 
dataYT['handsQt'] = dataYT['likes'] + dataYT['dislikes']
dataYT['likeRatio'] = dataYT.apply(lambda x: divNum(x['likes'], x['handsQt']), axis=1)
dataYT['dislikeRatio'] = dataYT.apply(lambda x: divNum(x['dislikes'], x['handsQt']), axis=1)
dataYT['dislikePerView'] = dataYT.apply(lambda x: divNum(x['dislikes'], x['views']), axis=1)
dataYT['likePerView'] = dataYT.apply(lambda x: divNum(x['likes'], x['views']), axis=1)
dataYT['handsPerView'] = dataYT.apply(lambda x: divNum(x['handsQt'], x['views']), axis=1)
dataYT['viewsPerDay'] = dataYT.apply(lambda x: divNum(x['views'], x['dateDays']), axis=1)
dataYT['handsPerDay'] = dataYT.apply(lambda x: divNum(x['handsQt'], x['dateDays']), axis=1)
dataYT['viewsPerSubscribers'] = dataYT.apply(lambda x: divNum(x['views'], x['subscribers']), axis=1)



In [6]:
# sorting dataset by singular run/simulation
dataYT = dataYT.sort_values(by=['search', 'setting', 'preaction', 'response','date','filename', 'ord'])
dataYT.reset_index(inplace=True)

# creating index column
dataYT['ind'] = dataYT.index

# getting sequence using index 
minInd = dataYT.groupby(['search', 'setting', 'preaction', 'response','date','filename'])['ind'].min()
minInd = pd.DataFrame(minInd)
minInd.reset_index(inplace=True)
minInd.columns = ['search', 'setting', 'preaction', 'response','date','filename', 'minind']
dataYT = pd.merge(dataYT, minInd, on=['search', 'setting', 'preaction', 'response','date','filename'], how='left')
dataYT['seq'] = dataYT['ind']-dataYT['minind']+1
dataYT = dataYT.drop(columns=['minind'])

# computing size of the sequences
maxInd = dataYT.groupby(['search', 'setting', 'preaction', 'response','date','filename'])['seq'].max()
maxInd = pd.DataFrame(maxInd)
maxInd.reset_index(inplace=True)
maxInd.columns = ['search', 'setting', 'preaction', 'response','date','filename', 'seq_max']
dataYT = pd.merge(dataYT, maxInd, on=['search', 'setting', 'preaction', 'response','date','filename'], how='left')

In [7]:
# Selecting sequences by lenght (here only sequences of lenght 20)
dataYT = dataYT[dataYT['seq_max'] > 19]
dataYT = dataYT[dataYT['seq'] < 21]

# Selecting just relevant executions/simulations
dataYT = dataYT[(dataYT.response == 'top movie') | ((dataYT.response == 'bottom movie'))]
dataYT = dataYT[(dataYT.preaction == "browsing news") | (dataYT.preaction == "none")]

In [8]:
# Creating bins for the movie sequence
dataYT['bin'] = dataYT.apply(lambda x: int(int(x['seq']-1)/BINSIZE)+1, axis=1)


In [9]:
# Creating time division after and before YouTube's policy change
dataYT['date'] = pd.to_datetime(dataYT['date'])
dataYT['pol'] = np.where(dataYT['date'] < pd.to_datetime("2019-02-1"), 'before', 'after')

In [10]:
# Printing all data columns
dataYT.columns

Index(['index', 'Unnamed: 0', 'category', 'channel', 'channel_id', 'clar_link',
       'clarify', 'date', 'datePublish', 'depth', 'dislikes', 'filename', 'id',
       'id,title,channel,channel_id,classification', 'key', 'lenght',
       'license', 'likes', 'order', 'preaction', 'rec_qt', 'recommendations',
       'response', 'search', 'setting', 'subscribers', 'title', 'verified',
       'views', 'classification', 'ord', 'verify', 'dateDays', 'clas',
       'handsQt', 'likeRatio', 'dislikeRatio', 'dislikePerView', 'likePerView',
       'handsPerView', 'viewsPerDay', 'handsPerDay', 'viewsPerSubscribers',
       'ind', 'seq', 'seq_max', 'bin', 'pol'],
      dtype='object')

In [11]:
# Getting data sequence average per bin
dataSeq = dataYT.groupby(['search', 'setting', 'preaction', 'response','date', 'filename', 'pol', 'bin'])['clas'].mean().unstack().fillna(0)
dataSeq = pd.DataFrame(dataSeq)
dataSeq.reset_index(inplace=True)
dataSeq.head()

bin             search setting      preaction      response       date  \
0    April the Giraffe  logged  browsing news  bottom movie 2018-10-27   
1    April the Giraffe  logged  browsing news  bottom movie 2018-10-30   
2    April the Giraffe  logged  browsing news  bottom movie 2018-11-09   
3    April the Giraffe  logged  browsing news  bottom movie 2018-12-22   
4    April the Giraffe  logged  browsing news  bottom movie 2019-01-30   

bin                                           filename     pol    1    2    3  \
0    data-browListNlL-April the Giraffe_2018-10-27_...  before -0.5 -0.5 -1.0   
1    data-browListNlL-April the Giraffe_2018-10-30_...  before -0.5 -1.0 -1.0   
2    data-browListNlL-April the Giraffe_2018-11-09_...  before  0.0  0.0 -0.5   
3    data-11browListNlL-April the Giraffe_2018-12-2...  before -0.5 -0.5 -0.5   
4    data-0browListNlL-April the Giraffe_2019-01-30...  before  0.0 -0.5  0.0   

bin    4    5    6    7    8    9   10  
0   -1.0 -1.0 -1.0 -1.0  0.0  0.5  0.0  
1   -0.5 -0.5 -0.5  0.5  0.0  0.0  0.5  
2    0.0  0.0 -0.5 -1.0 -1.0  0.0  0.0  
3   -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
4   -0.5  0.0  0.0 -0.5 -0.5  0.0  0.0

In [12]:
###
# Sampling for each query: 'setting', 'response'
# Getting minimum number of sequences per setting to giving query to sample
###

total = 0
for key, group in dataSeq.groupby('search'):
    
    # Getting mininum value to a giving query
    MINV = group.groupby(['setting', 'response']).size().min()
    print(key + " " + str(MINV))
    
    # creating group ids
    groupid = dataSeq.groupby(['setting',  'response']).size()
    groupid = pd.DataFrame(groupid)
    groupid.reset_index(inplace=True) 
    groupid.columns = ['setting',  'response', 'size']
    groupid['groupid'] = groupid.index
    groupid
    
    # Merging groupid with dataseq 
    dataSeqGroup = pd.merge(group, groupid, on=['setting',  'response'], how='left')
    
    # sampling 
    dataSizeGroup = dataSeqGroup.groupby('groupid').apply(lambda x: x.sample(MINV)).reset_index(drop=True)
    dataSizeGroup = dataSizeGroup.drop(columns=['size', 'groupid'])
    if total == 0:
            dataSameSizeGroup = dataSizeGroup.copy()
            total += MINV
    else:
            dataSameSizeGroup = dataSameSizeGroup.append(dataSizeGroup)
            total += MINV

# Merging with initialy parsed data and saving to file
dataYT_Sample1 = pd.merge(dataYT, dataSameSizeGroup[['search', 'setting', 'response','date', 'filename']], on=['search', 'setting', 'response','date', 'filename'], how='inner')
#dataYT_Sample1.to_csv('data/dataSampledQuerySetRes.csv', index=False)            

April the Giraffe 17
Bitcoin Price 20
DACA 18
Hurricane Harvey 33
Hurricane Irma 38
Hurricane Jose 24
Hurricane Maria 19
Las Vegas shooting 44
North Korea 24
Solar Eclipse 19


In [13]:
dataYT_Sample01 = dataYT_Sample1[dataYT_Sample1.response == 'top movie']
#dataYT_Sample01.to_csv('data/dataSampledQuerySetTop.csv', index=False) 

In [14]:
###
# Sampling for each query: 'setting', 'response', 'pol' (YouTube policy)
# Getting minimum number of sequences per setting to giving query to sample
###

total = 0
for key, group in dataSeq.groupby('search'):
    
    # Getting mininum value to a giving query
    MINV = group.groupby(['setting', 'response', 'pol']).size().min()
    print(key + " " + str(MINV))
    
    # creating group ids
    groupid = dataSeq.groupby(['setting',  'response', 'pol']).size()
    groupid = pd.DataFrame(groupid)
    groupid.reset_index(inplace=True) 
    groupid.columns = ['setting',  'response', 'pol', 'size']
    groupid['groupid'] = groupid.index
    groupid
    
    # Merging groupid with dataseq 
    dataSeqGroup = pd.merge(group, groupid, on=['setting',  'response', 'pol'], how='left')
    
    # sampling 
    dataSizeGroup = dataSeqGroup.groupby('groupid').apply(lambda x: x.sample(MINV)).reset_index(drop=True)
    dataSizeGroup = dataSizeGroup.drop(columns=['size', 'groupid'])
    if total == 0:
            dataSameSizeGroup = dataSizeGroup.copy()
            total += MINV
    else:
            dataSameSizeGroup = dataSameSizeGroup.append(dataSizeGroup)
            total += MINV

# Merging with initialy parsed data and saving to file
dataYT_Sample2 = pd.merge(dataYT, dataSameSizeGroup[['search', 'setting', 'response','date', 'filename', 'pol']], on=['search', 'setting', 'response','date', 'filename', 'pol'], how='inner')
#dataYT_Sample2.to_csv('data/dataSampledQuerySetResPol.csv', index=False)
print(len(dataYT_Sample2.index))

April the Giraffe 3
Bitcoin Price 7
DACA 9
Hurricane Harvey 9
Hurricane Irma 9
Hurricane Jose 7
Hurricane Maria 9
Las Vegas shooting 14
North Korea 7
Solar Eclipse 7
25920
